In [1]:
import sys
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

## Load DMS and store object

In [2]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)

In [3]:
print(dml.data_A.shape)
print(dml.data_B.shape)
print(dml.data_C.shape)

(34061, 47)
(32819, 47)
(26071, 47)


## Locating constant pvm y-intervals
Dataset B and C has 'holes' in y_train, where pv_measurement is constant. Therefore we don't want to train on this data and can remove the corresponding rows for both X and Y

In [4]:
y_hole_length = 30
from helpers import find_const_interval

In [13]:

#print(dml.data_B['pv_measurement'][19608])
dml.data_B.to_csv('data_B.csv')


In [5]:
# TAKES A WHILE TO RUN
# y_train_a_const_idx, ca = find_const_interval(dml.data_A, 'pv_measurement', y_hole_length)
# print('y_train_a anomalies:',ca)
y_train_b_const_idx, cb = find_const_interval(dml.data_B, 'pv_measurement', y_hole_length)
print('y_train_b anomalies:',cb)
y_train_c_const_idx, cc = find_const_interval(dml.data_C, 'pv_measurement', y_hole_length)
print('y_train_c anomalies:',cc)


326
327
444
445
975
976
1961
1962
3606
3607
3608
7210
7211
7212
7213
7214
8025
8028
8029
8170
8171
8172
8173
8174
8434
8435
8436
8437
8438
8602
8603
8604
8605
8606
8794
8795
8796
8797
8798
8843
8844
8845
8846
9320
9321
9324
9610
9611
9612
9613
9614
10045
10046
10047
10048
10049
10810
10811
10812
10813
10965
10971
13408
13409
13410
13411
13412
13413
15176
15177
15178
15179
15180
17166
17171
17172
17173
17174
17406
17414
17742
17748
17749
17750
17981
17988
17989
18056
18059
18060
18061
18062
18063
18064
18200
18203
18204
18205
18206
18207
18208
18251
18255
18299
18688
18689
18697
19145


KeyError: 19608

### Deleting the rows where pvm is const

In [32]:
train_a = dml.data_A
train_b = dml.data_B
train_c = dml.data_C

In [34]:
train_a.shape

(0, 0)

In [19]:
date_forecast_a_const = train_a.iloc[y_train_a_const_idx]['date_forecast']
date_forecast_a_const_values = date_forecast_a_const.values
train_a = train_a[~train_a['date_forecast'].isin(date_forecast_a_const_values)]

date_forecast_b_const = train_b.iloc[y_train_b_const_idx]['date_forecast']
date_forecast_b_const_values = date_forecast_b_const.values
train_b = train_b[~train_b['date_forecast'].isin(date_forecast_b_const_values)]

date_forecast_c_const = train_c.iloc[y_train_c_const_idx]['date_forecast']
date_forecast_c_const_values = date_forecast_c_const.values
train_c = train_c[~train_c['date_forecast'].isin(date_forecast_c_const_values)]


KeyError: 'date_forecast'

In [7]:

dml.data_A = train_a
dml.data_B = train_b
dml.data_C = train_c

### Mean value

In [4]:
%store dml

Stored 'dml' (Data_Manager)
